In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data)
student_data = pd.read_csv(student_data)

In [4]:
# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
# make sure that we have a complete data set
school_data_complete_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
dtype: int64

In [6]:
school_types_index = school_data_complete_df.set_index(["school_name"])["type"]

In [7]:
# create a variable to store each unique school
unique_schools = school_data_complete_df["school_name"].unique()
# the number of schools in the file equals the length of the unique schools
no_schools = len(unique_schools)
# total students equals the amount of rows from the csvfile 
total_students = len(school_data_complete_df["student_name"])
# create a dataframe grouped buy school name to be able to sum the total budget
schools_group_df = school_data_complete_df.groupby(["school_name"])
school_totals_df = schools_group_df.mean()
# total budget equals sum of the budget column
total_budget = school_totals_df.sum()["budget"]
# average maths and reading score equals mean of the math and reading score columns
avg_math = school_data_complete_df["math_score"].mean()
avg_reading = school_data_complete_df["reading_score"].mean()
# set the passing limit
limit = 70
# count the number of students who got a score of equal or greater than the limit
math_pass = (school_data_complete_df["math_score"] >= limit).sum()
read_pass = (school_data_complete_df["reading_score"] >= limit).sum()
# calculate the percent of people who passed maths/ reading
math_pass_perc = (math_pass/total_students) * 100
read_pass_perc = (read_pass/total_students) * 100
# overall pass means that the student passed both maths and reading
overall_pass = school_data_complete_df[(school_data_complete_df["math_score"] >= limit) &
                                       (school_data_complete_df["reading_score"] >= limit)]["student_name"].count()
# calculate the percentage of people who passed both subjects
overall_pass_perc = (overall_pass/total_students) * 100


In [8]:
# create a dataframe to store the district summary variables under the appropriate titles
district_summ_df = pd.DataFrame(
    {"Total Schools": [no_schools],
     "Total Students": [total_students],
     "Total Budget": [total_budget],
     "Average Math Score": [avg_math],
     "Average Reading Score": [avg_reading],
     "% Passing Math": [math_pass_perc],
     "% Passing Reading": [read_pass_perc],
     "% Overall Passing": [overall_pass_perc]
     })
district_summ_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,65.172326


In [9]:
# work out the total students per school by counting the amount of rows for each school
total_students_school = school_data_complete_df["school_name"].value_counts()
# total budget equals the average budget by school name
total_budget_school = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# per student budget equals the total of each school divided by the total amount of students per school
per_student_budg = total_budget_school / total_students_school
# average math/reading score equals the mean of the math/reading school by school
avg_math_score = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
avg_read_score = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]
# school type equals the type of school for each school
school_type = school_data.set_index(["school_name"])["type"]
# find all the students who passed maths/ reading
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
# count the amount of students who passed math/reading for each school
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]
# calculate this as a percentage of the total students at each school
per_school_passing_math = per_school_passing_math / total_students_school * 100
per_school_passing_reading = per_school_passing_reading / total_students_school * 100
# calculate the amount of people who passed both maths & reading
per_school_overall_pass = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]
# count the amount of students for each school
per_school_overall_pass = per_school_overall_pass.groupby(["school_name"]).count()["student_name"]
# work out the percentage of overall pass rate by dividing by the total number of students at each school
per_school_overall_pass_perc = per_school_overall_pass / total_students_school * 100


In [10]:
# create a school summary dataframe to 
school_sum_df = pd.DataFrame(
    {"School Type": school_type,
     "Total Students": total_students_school,
     "Total School Budget": total_budget_school,
     "Budget Per Student": per_student_budg,
     "Average Math Score": avg_math_score,
     "Average Reading Score": avg_read_score,
     "% Passing Math": per_school_passing_math,
     "% Passing reading": per_school_passing_reading,
     "% Overall Passing": per_school_overall_pass_perc
     
})
# use mapping to format the data
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
school_sum_df["Budget Per Student"] = school_sum_df["Budget Per Student"].map("${:,.2f}".format)
school_sum_df.head(16)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [11]:
# create a dataframe to highlight the top 5 performing schools by overall passing %
top_5_overall_pass_df = school_sum_df.sort_values("% Overall Passing", ascending=False)
top_5_overall_pass_df.head(6)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333


In [12]:
# create a dataframe to highlight the bottom 5 performing schools by overall passing %
bottom_5_overall_pass_df = school_sum_df.sort_values("% Overall Passing")
bottom_5_overall_pass_df.head(6)

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887


In [13]:
# create variable for grades
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]
# calculate the mean math score for each grade
ninth_graders_math_score = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_math_score = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_math_score = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_math_score = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [14]:
# store results in a dataframe
maths_score_grade_df = pd.DataFrame(
    {"9th": ninth_graders_math_score,
     "10th": tenth_graders_math_score,
     "11th": eleventh_graders_math_score,
     "12th": twelfth_graders_math_score
    })
maths_score_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [15]:
# Calculate the mean reading score for each grade
ninth_graders_reading_score = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_reading_score = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_reading_score = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_reading_score = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [16]:
# store results in a dataframe
reading_score_grade_df = pd.DataFrame(
    {"9th": ninth_graders_reading_score,
     "10th": tenth_graders_reading_score,
     "11th": eleventh_graders_reading_score,
     "12th": twelfth_graders_reading_score
    })
reading_score_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [25]:
# create bins for each interval 
bins = [0, 585, 630, 645, 680]

# create the labels which each bin will be assigned to
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a new df which is a copy of the school summary
new_df = school_sum_df.copy()
# Assign a column to store the spending ranges results
new_df["Spending Ranges (Per Student)"] = pd.cut(per_student_budg, bins, labels=labels, right=False)


In [18]:
# calculate the total students budget
total_students_budg = new_df["Spending Ranges (Per Student)"].value_counts()
# calculate the average maths and reading score
avg_budg_math = new_df.groupby("Spending Ranges (Per Student)").mean()["Average Math Score"]
avg_budg_reading = new_df.groupby("Spending Ranges (Per Student)").mean()["Average Reading Score"]
# calculate the perc passing maths and reading score
perc_pass_bugd_math_perc = new_df.groupby("Spending Ranges (Per Student)").mean()["% Passing Math"]
perc_pass_bugd_reading_perc = new_df.groupby("Spending Ranges (Per Student)").mean()["% Passing reading"]
# calculate the perc overall passing score
per_budg_overall_pass_perc = new_df.groupby("Spending Ranges (Per Student)").mean()["% Overall Passing"]

In [28]:
# store results in a database and format to 2 decimals
budget_df = pd.DataFrame(
    {"Average Maths Score": avg_budg_math.map("{:.2f}".format),
     "Average Reading Score": avg_budg_reading.map("{:.2f}".format),
     "% Passing Math": perc_pass_bugd_math_perc.map("{:.2f}".format),
     "% Passing Reading": perc_pass_bugd_reading_perc.map("{:.2f}".format),
     "% Overall Passing": per_budg_overall_pass_perc.map("{:.2f}".format)
    })
budget_df

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [20]:
# create bins for each interval
bins1 = [0, 1000, 2000, 5000]
# create the labels which each bin will be assigned to
labels1 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# add school size to the new_df
new_df["School Size"] = new_df["Total Students"]
new_df["School Size"] = pd.cut(new_df["Total Students"], bins1, labels=labels1, right=False)


In [21]:
# calculate the total students size
total_students_size = new_df["School Size"].value_counts()
# calculate the average maths and reading score
avg_size_math = new_df.groupby("School Size").mean()["Average Math Score"]
avg_size_reading = new_df.groupby("School Size").mean()["Average Reading Score"]
# calculate the perc passing maths and reading score
perc_pass_size_math_perc = new_df.groupby("School Size").mean()["% Passing Math"]
perc_pass_size_reading_perc = new_df.groupby("School Size").mean()["% Passing reading"]
# calculate the perc overall passing score
per_size_overall_pass_perc = new_df.groupby("School Size").mean()["% Overall Passing"]


In [22]:
# store results in a database
size_df = pd.DataFrame(
    {"Average Maths Score": avg_size_math,
     "Average Reading Score": avg_size_reading,
     "% Passing Math": perc_pass_size_math_perc,
     "% Passing Reading": perc_pass_size_reading_perc,
     "% Overall Passing": per_size_overall_pass_perc
    })
size_df

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [23]:
# calculate the total students size
total_students_type = new_df["School Type"].value_counts()
# calculate the average maths and reading score
avg_type_math = new_df.groupby("School Type").mean()["Average Math Score"]
avg_type_reading = new_df.groupby("School Type").mean()["Average Reading Score"]
# calculate the perc passing maths and reading score
perc_pass_type_math_perc = new_df.groupby("School Type").mean()["% Passing Math"]
perc_pass_type_reading_perc = new_df.groupby("School Type").mean()["% Passing reading"]
# calculate the perc overall passing score
per_type_overall_pass_perc = new_df.groupby("School Type").mean()["% Overall Passing"]


In [24]:
# store results in a database
type_df = pd.DataFrame(
    {"Average Maths Score": avg_type_math,
    "Average Reading Score": avg_type_reading,
    "% Passing Math": perc_pass_type_math_perc,
    "% Passing Reading": perc_pass_type_reading_perc,
     "% Overall Passing": per_type_overall_pass_perc
    })
type_df

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
